In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [14]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255
m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 13000
The number of features = 784


In [15]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [16]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [72]:
lr = 0.1
arch_test = {1,5,10,50,100}
arch = [100] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [73]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 130


In [74]:
#Theta Initialization 
def theta_init:
    theta = []

    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
        return theta

In [75]:
def activation(x):
    return 1/(1+np.exp(-x))

In [76]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    #l1 = activation(np.dot(l0, theta[0]))
    #l2 = activation(np.dot(l1, theta[1]))
    return fm

In [77]:
fm = forward_prop(X_train, theta)

In [78]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [79]:
cost_total(X_train, theta, train_class_enc, m)

4.312642589139592

In [80]:
epoch = 0
while(True):
    count = 0

    cost_init = cost_final
    print("Initial Cost for this epoch {} = {}".format(epoch, cost_init))
    
    for b in mini_batch:
        X_b = b[0]
        Y_b = b[1]
        fm = forward_prop(X_b, theta)
        delta = [None]*len(fm)
        
        if (count % 60 == 0):
            print("Error = "+str(cost_total(X_b, theta, Y_b, batch_size)))
        #Backward Propagation
        
        for l in range(len(fm)-1, 0, -1):
            if (l == len(fm)-1):
                delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
            else:
                delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

        #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

        for t in range(len(theta)):
            theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 
        
        count+=1
    epoch+=1 #Number of epochs
    cost_final = cost_total(X_train, theta, train_class_enc, m)
    print("Cost after {} epochs is = {}".format(epoch, cost_final))
    if (abs(cost_final-cost_init) < 0.00000001):
        print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
        break

Initial Cost for this epoch 0 = 0.48077266410059316
Error = 4.245461022322803
Error = 1.0847412438329622
Error = 0.9789941262376965
Cost after 1 epochs is = 0.9817071706329997
Initial Cost for this epoch 1 = 0.9817071706329997
Error = 0.97268352962039
Error = 0.9914286617876473
Error = 0.9524972598075686
Cost after 2 epochs is = 0.9621633078961342
Initial Cost for this epoch 2 = 0.9621633078961342
Error = 0.9525634515448745
Error = 0.9827638360551145
Error = 0.9448794319486096
Cost after 3 epochs is = 0.9564515442477083
Initial Cost for this epoch 3 = 0.9564515442477083
Error = 0.9478430415840039
Error = 0.9796512577607647
Error = 0.9397430648347614
Cost after 4 epochs is = 0.9521363552723612
Initial Cost for this epoch 4 = 0.9521363552723612
Error = 0.9446115767752081
Error = 0.9766217765731693
Error = 0.9269063479739197
Cost after 5 epochs is = 0.933641023630544
Initial Cost for this epoch 5 = 0.933641023630544
Error = 0.9278026010921554
Error = 0.5436639025389054
Error = 0.486533130

Error = 0.4250130093222819
Cost after 47 epochs is = 0.4271319317532007
Initial Cost for this epoch 47 = 0.4271319317532007
Error = 0.41901914469659607
Error = 0.42535012848812287
Error = 0.4234044551385412
Cost after 48 epochs is = 0.42560302772831254
Initial Cost for this epoch 48 = 0.42560302772831254
Error = 0.4168933751566338
Error = 0.4230768353060447
Error = 0.42178702687180647
Cost after 49 epochs is = 0.4240785882794851
Initial Cost for this epoch 49 = 0.4240785882794851
Error = 0.41476919264007495
Error = 0.42082979314744484
Error = 0.4201685118465652
Cost after 50 epochs is = 0.42256175428535653
Initial Cost for this epoch 50 = 0.42256175428535653
Error = 0.41264794914278335
Error = 0.4186124462649419
Error = 0.4185565234794183
Cost after 51 epochs is = 0.4210545530724323
Initial Cost for this epoch 51 = 0.4210545530724323
Error = 0.41053033623389923
Error = 0.41642856411960166
Error = 0.4169584098427934
Cost after 52 epochs is = 0.4195581042720528
Initial Cost for this epoc

Error = 0.34854332437981894
Cost after 93 epochs is = 0.35889959567680385
Initial Cost for this epoch 93 = 0.35889959567680385
Error = 0.32420018723871025
Error = 0.34862981243996743
Error = 0.34678745362704905
Cost after 94 epochs is = 0.3574910585903219
Initial Cost for this epoch 94 = 0.3574910585903219
Error = 0.3227422390454629
Error = 0.34739298584187184
Error = 0.34504507425503184
Cost after 95 epochs is = 0.3560662740080681
Initial Cost for this epoch 95 = 0.3560662740080681
Error = 0.3214480516202684
Error = 0.3461150210349895
Error = 0.3433225304808902
Cost after 96 epochs is = 0.35461973710561906
Initial Cost for this epoch 96 = 0.35461973710561906
Error = 0.32030622874403347
Error = 0.34477867330347267
Error = 0.3416219931059527
Cost after 97 epochs is = 0.3531472517328711
Initial Cost for this epoch 97 = 0.3531472517328711
Error = 0.3192780287117212
Error = 0.3433721945636458
Error = 0.3399366610902543
Cost after 98 epochs is = 0.3516479062979665
Initial Cost for this epoc

Cost after 138 epochs is = 0.3080420699564186
Initial Cost for this epoch 138 = 0.3080420699564186
Error = 0.2766653983679535
Error = 0.2965537064051898
Error = 0.2791258803703607
Cost after 139 epochs is = 0.30736489160402075
Initial Cost for this epoch 139 = 0.30736489160402075
Error = 0.2761210483038882
Error = 0.2958586433735024
Error = 0.2783899552060214
Cost after 140 epochs is = 0.3066981921654108
Initial Cost for this epoch 140 = 0.3066981921654108
Error = 0.2755917603563171
Error = 0.2951798633824964
Error = 0.27767375148643364
Cost after 141 epochs is = 0.3060413501971502
Initial Cost for this epoch 141 = 0.3060413501971502
Error = 0.27507654101921264
Error = 0.2945163550109878
Error = 0.27697591028680413
Cost after 142 epochs is = 0.3053937374407945
Initial Cost for this epoch 142 = 0.3053937374407945
Error = 0.27457448035314835
Error = 0.2938670956292912
Error = 0.2762951046862178
Cost after 143 epochs is = 0.30475471302742363
Initial Cost for this epoch 143 = 0.30475471302

Error = 0.2467650765963167
Cost after 184 epochs is = 0.2656774783518014
Initial Cost for this epoch 184 = 0.2656774783518014
Error = 0.23693490491842809
Error = 0.25367191686462287
Error = 0.2461130403254898
Cost after 185 epochs is = 0.26483585908702056
Initial Cost for this epoch 185 = 0.26483585908702056
Error = 0.23618197528837287
Error = 0.25296152458376336
Error = 0.24548461067417798
Cost after 186 epochs is = 0.2640275759273853
Initial Cost for this epoch 186 = 0.2640275759273853
Error = 0.23547605539216446
Error = 0.2522731271049118
Error = 0.24487839818174678
Cost after 187 epochs is = 0.2632497091849627
Initial Cost for this epoch 187 = 0.2632497091849627
Error = 0.23480970333848727
Error = 0.2516044713749235
Error = 0.24429319518883844
Cost after 188 epochs is = 0.2624997205291816
Initial Cost for this epoch 188 = 0.2624997205291816
Error = 0.2341767448486283
Error = 0.2509538419172803
Error = 0.24372796840151636
Cost after 189 epochs is = 0.261775383917301
Initial Cost for

...

Error = 0.027051007843412064
Cost after 9158 epochs is = 0.016083955358783802
Initial Cost for this epoch 9158 = 0.016083955358783802
Error = 0.026263804847148516
Error = 0.011399598506636351
Error = 0.027050568634763628
Cost after 9159 epochs is = 0.016083384991650587
Initial Cost for this epoch 9159 = 0.016083384991650587
Error = 0.02626354857411956
Error = 0.011399105900311679
Error = 0.027050129755255416
Cost after 9160 epochs is = 0.016082815103934795
Initial Cost for this epoch 9160 = 0.016082815103934795
Error = 0.026263292226908735
Error = 0.01139861321728193
Error = 0.027049691206566605
Cost after 9161 epochs is = 0.01608224568866286
Initial Cost for this epoch 9161 = 0.01608224568866286
Error = 0.026263035806846906
Error = 0.011398120457005905
Error = 0.027049252990359623
Cost after 9162 epochs is = 0.0160816767389462
Initial Cost for this epoch 9162 = 0.0160816767389462
Error = 0.026262779315251276
Error = 0.011397627618937912
Error = 0.02704881510828124
Cost after 9163 epoc

Error = 0.027032014900269616
Cost after 9202 epochs is = 0.0160592304109622
Initial Cost for this epoch 9202 = 0.0160592304109622
Error = 0.026252475018678972
Error = 0.011377843563781498
Error = 0.027031591645363486
Cost after 9203 epochs is = 0.016058675484793516
Initial Cost for this epoch 9203 = 0.016058675484793516
Error = 0.02625221662903803
Error = 0.011377347018090313
Error = 0.027031168786748282
Cost after 9204 epochs is = 0.016058120794661204
Initial Cost for this epoch 9204 = 0.016058120794661204
Error = 0.026251958217029553
Error = 0.011376850368020736
Error = 0.02703074632589746
Cost after 9205 epochs is = 0.01605756633621201
Initial Cost for this epoch 9205 = 0.01605756633621201
Error = 0.026251699783763193
Error = 0.011376353612831071
Error = 0.02703032426428309
Cost after 9206 epochs is = 0.016057012105133314
Initial Cost for this epoch 9206 = 0.016057012105133314
Error = 0.02625144133034792
Error = 0.011375856751772898
Error = 0.0270299026033757
Cost after 9207 epochs 

Error = 0.027013785712918736
Cost after 9246 epochs is = 0.016034984277223628
Initial Cost for this epoch 9246 = 0.016034984277223628
Error = 0.026241099360659118
Error = 0.011355885926143512
Error = 0.027013381245483293
Cost after 9247 epochs is = 0.01603443609191585
Initial Cost for this epoch 9247 = 0.01603443609191585
Error = 0.026240841032694515
Error = 0.011355383978661314
Error = 0.027012977235261523
Cost after 9248 epochs is = 0.01603388798351616
Initial Cost for this epoch 9248 = 0.01603388798351616
Error = 0.02624058273102487
Error = 0.01135488188603108
Error = 0.02701257368346126
Cost after 9249 epochs is = 0.016033339949061115
Initial Cost for this epoch 9249 = 0.016033339949061115
Error = 0.026240324456762467
Error = 0.011354379647090396
Error = 0.027012170591277744
Cost after 9250 epochs is = 0.016032791985612117
Initial Cost for this epoch 9250 = 0.016032791985612117
Error = 0.02624006621102039
Error = 0.011353877260664334
Error = 0.027011767959893428
Cost after 9251 epo

Error = 0.02699643556291856
Cost after 9290 epochs is = 0.016010900838309094
Initial Cost for this epoch 9290 = 0.016010900838309094
Error = 0.026229772696108846
Error = 0.011333645648891548
Error = 0.02699605213834328
Cost after 9291 epochs is = 0.01601035354326778
Initial Cost for this epoch 9291 = 0.01601035354326778
Error = 0.026229516594698604
Error = 0.011333136025446302
Error = 0.026995669207146564
Cost after 9292 epochs is = 0.016009806216568587
Initial Cost for this epoch 9292 = 0.016009806216568587
Error = 0.026229260569988526
Error = 0.011332626191354256
Error = 0.026995286769602857
Cost after 9293 epochs is = 0.016009258856247783
Initial Cost for this epoch 9293 = 0.016009258856247783
Error = 0.02622900462318186
Error = 0.01133211614470067
Error = 0.026994904825954697
Cost after 9294 epochs is = 0.01600871146036486
Initial Cost for this epoch 9294 = 0.01600871146036486
Error = 0.026228748755485656
Error = 0.011331605883546545
Error = 0.02699452337641234
Cost after 9295 epoc

KeyboardInterrupt: 

In [81]:
pred_class = forward_prop(X_test, theta)
test_pred_class = pred_class[-1]
for i in range(len(test_pred_class)):
    test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
    test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0
    
    
test_acc = 0
for i in range(len(test_actual_class_enc)):
    if (np.array_equal(test_pred_class[i], test_actual_class_enc[i])):
        test_acc+=1
test_acc /= X_test.shape[0]

print("The Test Accuracy of the model = {}%".format(test_acc*100))

The Test Accuracy of the model = 88.07692307692308%


In [83]:
theta

[array([[ 0.58134227, -1.02686217,  0.25625017, ..., -1.16576651,
          1.1661437 ,  0.03576611],
        [ 0.45131894, -0.88210178,  0.95926732, ..., -0.4563415 ,
         -0.81666037, -0.64694446],
        [ 0.64151183,  0.9239467 ,  0.02900217, ...,  0.07335058,
         -0.79886324,  0.03394589],
        ...,
        [ 0.54901955, -0.44196044,  0.95803006, ...,  0.18557725,
         -0.20419956, -0.44436999],
        [ 0.81220314,  0.88073405, -0.21869598, ...,  0.69237579,
         -0.3110474 ,  0.6972845 ],
        [-0.24377097,  0.71378875,  0.63720245, ...,  0.23274569,
         -0.33735958, -0.45779068]]),
 array([[ 1.8853407 , -1.03470059,  1.01753464, ..., -1.76709672,
         -1.41290283, -1.89219375],
        [-0.68788815,  2.0245168 ,  1.94531416, ..., -1.16763619,
         -0.36193954, -2.05250739],
        [-1.96426142, -0.8567153 ,  0.88017553, ..., -1.29227659,
          0.82155758, -0.51576832],
        ...,
        [-1.44467032, -0.84938886,  0.43479276, ...,  